In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

chrome_driver_path = "E:\Liga 1 Fix\chromedriver-win64\chromedriver.exe"

link_klub = pd.read_csv('Link_Klub.csv')
link_klub = link_klub['Link Klub'].tolist()
for link in link_klub[:2]:

    # Setup Driver
    options = webdriver.ChromeOptions()
    #options.add_argument("--headless") # Biar Chrome-nya tidak dibuka (Jalan di belakang)
    service = Service(chrome_driver_path)
    driver = webdriver.Chrome(service=service, options=options)
    driver.maximize_window() # Kalo headless, ini nggak berguna

    driver.get(link)

    pola = r'(?<=/single/BRI_LIGA_1_2024-25/)([A-Za-z0-9_]+)'
    nama_klub = re.search(pola, link).group(1)
    print("Scrap link klub: ", link)

    time.sleep(3)

    try:
        xpath_tab_pemain = '/html/body/div[2]/section/div/div/div/div[1]/ul/li[2]/a' # Klik Menu Pemain
        tab_pemain = driver.find_element(By.XPATH, xpath_tab_pemain)
        tab_pemain.click()
        time.sleep(3)
    except Exception as e:
        print('Gagal Klik tab pemain: ', e) # Nampilin Jenis ERRORNYA (KALAU ERROR)
        driver.quit() # Close Chrome
        exit()

    links_pemain = []
    i = 1
    while True:
        try:
            xpath_pemain = f"/html/body/div[2]/section/div/div/div/div[2]/div/div[2]/div/div[{i}]/div/a"
            element_pemain = driver.find_element(By.XPATH, xpath_pemain)
            href = element_pemain.get_attribute("href")
            if href:
                links_pemain.append(href)
            i += 1
        except:
            break # Stop kalau elemen tidak ditemukan lagi

    data = pd.DataFrame(links_pemain, columns=['Link Pemain'])
    data.to_csv(f'sementara-Link Pemain/{nama_klub}.csv', index=False, encoding='utf-8-sig')
    print(f"Total {len(links_pemain)} link pemain berhasil di simpan ke '{nama_klub}.csv")

driver.quit()

Scrap link klub:  https://ligaindonesiabaru.com/clubs/single/BRI_LIGA_1_2024-25/AREMA_FC
Total 30 link pemain berhasil di simpan ke 'AREMA_FC.csv
Scrap link klub:  https://ligaindonesiabaru.com/clubs/single/BRI_LIGA_1_2024-25/BALI_UNITED_FC
Total 38 link pemain berhasil di simpan ke 'BALI_UNITED_FC.csv
